# Exception Handling

Python has built-in exception handling you can use with Try/Except statements. PyEZ predefined exceptions are located in `jnpr.junos.exception`.

You first need to import the exceptions.You can import individual exceptions like this:

```python
# Device errors
from jnpr.junos.exception import ConnectError
from jnpr.junos.exception import ConnectTimeoutError
from jnpr.junos.exception import ConnectAuthError
from jnpr.junos.exception import ConnectRefusedError
# Config errors
from jnpr.junos.exception import LockError
from jnpr.junos.exception import UnlockError
from jnpr.junos.exception import ConfigLoadError
from jnpr.junos.exception import CommitError
```

You can also import all exceptions with:

```python
from jnpr.junos.exception import *
```

With all of the exceptions imported you can now take action on a per-exception basis.

## Example: Exception Handling

Here is an example where we will purposefully connect to a device on the wrong port to induce a `ConnectTimeoutError`.

Try changing the host/user/password/port values to generate different types of errors.

Hint: use 127.0.0.2 for a `ConnectRefusedError`.


In [ ]:
from jnpr.junos import Device
from jnpr.junos.exception import ConnectError
from jnpr.junos.exception import ConnectTimeoutError
from jnpr.junos.exception import ConnectAuthError
from jnpr.junos.exception import ConnectRefusedError

try:
    dev = Device(host='10.0.0.180', user='root', password='Juniper', port='222')
    dev.open()
except ConnectTimeoutError as err:
    print ("Timeout: {0}".format(err))
except ConnectAuthError as err:
    print ("Auth Error: {0}".format(err))
except ConnectRefusedError as err:
    print ("Refused: {0}".format(err))

## No exception handling
Compare the output above to the output of the cell below where there is no error handling:

In [ ]:
from jnpr.junos import Device
from jnpr.junos import exception

dev = Device(host='10.0.0.180', user='root', password='Juniper', port='222')
dev.open()


## Example: No exception handling

```python
---------------------------------------------------------------------------
SSHError                                  Traceback (most recent call last)
/opt/conda/lib/python3.5/site-packages/jnpr/junos/device.py in open(self, *vargs, **kvargs)
    875                 ssh_config=self._sshconf_lkup(),
--> 876                 device_params={'name': 'junos', 'local': False})
    877 

/opt/conda/lib/python3.5/site-packages/ncclient/manager.py in connect(*args, **kwds)
    152         else:
--> 153             return connect_ssh(*args, **kwds)
    154 

/opt/conda/lib/python3.5/site-packages/ncclient/manager.py in connect_ssh(*args, **kwds)
    117     try:
--> 118        session.connect(*args, **kwds)
    119     except Exception as ex:

/opt/conda/lib/python3.5/site-packages/ncclient/transport/ssh.py in connect(self, host, port, timeout, unknown_host_cb, username, password, key_filename, allow_agent, hostkey_verify, look_for_keys, ssh_config)
    368             else:
--> 369                 raise SSHError("Could not open socket to %s:%s" % (host, port))
    370 

SSHError: Could not open socket to 10.0.0.181:222

During handling of the above exception, another exception occurred:

ConnectRefusedError                       Traceback (most recent call last)
<ipython-input-13-90c69cb002fa> in <module>()
      3 
      4 dev = Device(host='10.0.0.181', user='root', password='Juniper', port='222')
----> 5 dev.open()

/opt/conda/lib/python3.5/site-packages/jnpr/junos/device.py in open(self, *vargs, **kvargs)
    891             diff_ts = ts_err - ts_start
    892             if diff_ts.seconds < 3:
--> 893                 raise EzErrors.ConnectRefusedError(self)
    894 
    895             # at this point, we assume that the connection

ConnectRefusedError: ConnectRefusedError(10.0.0.181)
```

## Catching all exceptions

Python's highest level exception is `BaseException`. Using this exception you can catch all other exceptions. This is useful if you want brevity of code (*read: lazy*).

In [ ]:
from jnpr.junos import Device
from jnpr.junos import exception
try:
    dev = Device(host='10.0.0.180', user='root', password='Juniper', port='222')
    dev.open()
except BaseException as err:
    print ("Cannot connect to device: {0}".format(err))

## Why Exception Handling

Without exception handling any exceptions which occur will generate unsightly output which may not make much sense to your user. 

However with exception handling you can display more appropriate messages and more importantly, now take action depending on the exception.

For example, if authentication fails you can prompt the user for the passwrod again. If locking the configuration database fails you can trigger a cooldown to wait for any existing config to be committed and try again.

## Full Example

This script was pulled from the PyEZ developer guide as an example of exception handling throughout the script.

```python
from jnpr.junos import Device
from jnpr.junos.utils.config import Config
from jnpr.junos.exception import ConnectError
from jnpr.junos.exception import LockError
from jnpr.junos.exception import UnlockError
from jnpr.junos.exception import ConfigLoadError
from jnpr.junos.exception import CommitError

host = 'dc1a.example.com'
conf_file = 'configs/junos-config-add-op-script.conf'


def main():
    # open a connection with the device and start a NETCONF session
    try:
        dev = Device(host=host)
        dev.open()
    except ConnectError as err:
        print ("Cannot connect to device: {0}".format(err))
        return

    dev.bind(cu=Config)

    # Lock the configuration, load configuration changes, and commit
    print ("Locking the configuration")
    try:
        dev.cu.lock()
    except LockError as err:
        print ("Unable to lock configuration: {0}".format(err))
        dev.close()
        return

    print ("Loading configuration changes")
    try:
        dev.cu.load(path=conf_file, merge=True)
    except (ConfigLoadError, Exception) as err:
        print ("Unable to load configuration changes: {0}".format(err))
        print ("Unlocking the configuration")
        try:
                dev.cu.unlock()
        except UnlockError:
            print ("Unable to unlock configuration: {0}".format(err))
        dev.close()
        return

    print ("Committing the configuration")
    try:
        dev.cu.commit(comment='Loaded by example.')
    except CommitError as err:
        print ("Unable to commit configuration: {0}".format(err))
        print ("Unlocking the configuration")
        try:
            dev.cu.unlock()
        except UnlockError as err:
            print ("Unable to unlock configuration: {0}".format(err))
        dev.close()
        return

    print ("Unlocking the configuration")
    try:
        dev.cu.unlock()
    except UnlockError as err:
        print ("Unable to unlock configuration: {0}".format(err))

    # End the NETCONF session and close the connection
    dev.close()

if __name__ == "__main__":
    main()
```